In [2]:
!echo "List of NVIDIA GPUs available on the system:"
!nvidia-smi --query-gpu=gpu_name --format=csv

List of NVIDIA GPUs available on the system:
zsh:1: command not found: nvidia-smi


In [3]:
!echo "NVIDIA GPU used by this notebook:"
!nvidia-smi --query-gpu=name --format=csv

NVIDIA GPU used by this notebook:
zsh:1: command not found: nvidia-smi


In [7]:
!{sys.executable} -m pip install --pre 'bigdl-llm[xpu]==2.4.0b20231116' --no-warn-script-location -f https://developer.intel.com/ipex-whl-stable-xpu > /dev/null

ERROR: Could not find a version that satisfies the requirement bigdl-llm==2.4.0b20231116 (from versions: 2.4.0b20230930, 2.4.0, 2.5.0b0, 2.5.0b1, 2.5.0b2, 2.5.0b20231201, 2.5.0b20231202, 2.5.0b20231203, 2.5.0b20231204, 2.5.0b20231205, 2.5.0b20231206, 2.5.0b20231207, 2.5.0b20231208, 2.5.0b20231209, 2.5.0b20231210, 2.5.0b20231211, 2.5.0b20231212, 2.5.0b20231213, 2.5.0b20231214, 2.5.0b20231215, 2.5.0b20231216, 2.5.0b20231217, 2.5.0b20231218, 2.5.0b20231219, 2.5.0b20231220, 2.5.0b20231221, 2.5.0b20231222, 2.5.0b20231223, 2.5.0b20231224, 2.5.0b20231225, 2.5.0b20231226, 2.5.0b20231227, 2.5.0b20231228, 2.5.0b20231229, 2.5.0b20231230, 2.5.0b20231231, 2.5.0b20240101, 2.5.0b20240103, 2.5.0b20240104, 2.5.0b20240105, 2.5.0b20240106, 2.5.0b20240107, 2.5.0b20240108, 2.5.0b20240109, 2.5.0b20240110, 2.5.0b20240111, 2.5.0b20240112, 2.5.0b20240113, 2.5.0b20240114, 2.5.0b20240115, 2.5.0b20240116, 2.5.0b20240117, 2.5.0b20240118, 2.5.0b20240122, 2.5.0b20240123, 2.5.0b20240124, 2.5.0b20240125, 2.5.0b2024012

In [13]:
import sys
import site
from pathlib import Path

!echo "Installation in progress, please wait..."
!{sys.executable} -m pip cache purge > /dev/null
!{sys.executable} -m pip install --pre bigdl-llm[xpu]==2.4.0b20231116 --no-warn-script-location -f https://developer.intel.com/ipex-whl-stable-xpu > /dev/null
!{sys.executable} -m pip install peft==0.5.0 --no-deps > /dev/null
!{sys.executable} -m pip install accelerate==0.23.0 --no-deps --no-warn-script-location > /dev/null
!{sys.executable} -m pip install  transformers==4.34.0 --no-warn-script-location > /dev/null 
!{sys.executable} -m pip install datasets==2.15 --no-warn-script-location > /dev/null 2>&1 
!{sys.executable} -m pip install fsspec==2023.9.2 > /dev/null  2>&1
!echo "Installation completed."

def get_python_version():
    return "python" + ".".join(map(str, sys.version_info[:2]))

def set_local_bin_path():
    local_bin = str(Path.home() / ".local" / "bin") 
    local_site_packages = str(
        Path.home() / ".local" / "lib" / get_python_version() / "site-packages"
    )
    sys.path.append(local_bin)
    sys.path.insert(0, site.getusersitepackages())
    sys.path.insert(0, sys.path.pop(sys.path.index(local_site_packages)))

set_local_bin_path()

Installation in progress, please wait...
zsh:1: no matches found: bigdl-llm[xpu]==2.4.0b20231116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.23.0 requires torch>=1.10.0, which is not installed.
peft 0.5.0 requires torch>=1.13.0, which is not installed.
datasets 2.15.0 requires huggingface-hub>=0.18.0, but you have huggingface-hub 0.17.3 which is incompatible.
Installation completed.


In [14]:
import logging
import os
import sys
from math import ceil
from typing import Optional, Tuple
import warnings

warnings.filterwarnings(
    "ignore", category=UserWarning, module="intel_extension_for_pytorch"
)
warnings.filterwarnings(
    "ignore", category=UserWarning, module="torchvision.io.image", lineno=13
)
warnings.filterwarnings("ignore", message="You are using the default legacy behaviour")
warnings.filterwarnings("ignore", category=UserWarning, message=".*Parameter.*")
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="This implementation of AdamW is deprecated",
)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["NUMEXPR_MAX_THREADS"] = "28"
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("bigdl").setLevel(logging.ERROR)


import torch
import intel_extension_for_pytorch as ipex
from datasets import load_dataset
from datasets import Dataset
from bigdl.llm.transformers import AutoModelForCausalLM
from bigdl.llm.transformers.qlora import (
    get_peft_model,
    prepare_model_for_kbit_training as prepare_model,
)
from peft import LoraConfig
from bigdl.llm.transformers.qlora import PeftModel
import transformers
from transformers import (
    DataCollatorForSeq2Seq,
    LlamaTokenizer,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

transformers.logging.set_verbosity_error()

ModuleNotFoundError: No module named 'torch'

In [ ]:
import os
import shutil

def check_disk_space(path="~/.cache/huggingface/"):
    abs_path = os.path.expanduser(path)
    total, used, free = shutil.disk_usage(abs_path)
    print(f"Total: {total // (2**30)} GiB")
    print(f"Used: {used // (2**30)} GiB")
    print(f"Free: {free // (2**30)} GiB")

# Example usage
check_disk_space()

In [ ]:
BASE_MODELS = {
    "0": "NousResearch/Nous-Hermes-Llama-2-7b",  # https://huggingface.co/NousResearch/Nous-Hermes-llama-2-7b
    "1": "NousResearch/Llama-2-7b-chat-hf",  # https://huggingface.co/NousResearch/Llama-2-7b-chat-hf
    "2": "NousResearch/Llama-2-13b-hf",  # https://huggingface.co/NousResearch/Llama-2-13b-hf
    "3": "NousResearch/CodeLlama-7b-hf",  # https://huggingface.co/NousResearch/CodeLlama-7b-hf
    "4": "Phind/Phind-CodeLlama-34B-v2",  # https://huggingface.co/Phind/Phind-CodeLlama-34B-v2
    "5": "openlm-research/open_llama_3b_v2",  # https://huggingface.co/openlm-research/open_llama_3b_v2
    "6": "openlm-research/open_llama_13b",  # https://huggingface.co/openlm-research/open_llama_13b
    "7": "HuggingFaceH4/zephyr-7b-beta", # https://huggingface.co/HuggingFaceH4/zephyr-7b-beta
}
BASE_MODEL = BASE_MODELS["3"]
DATA_PATH = "b-mc2/sql-create-context"
MODEL_PATH = "./final_model"
ADAPTER_PATH = "./lora_adapters"
DEVICE = torch.device("xpu" if torch.xpu.is_available() else "cpu")
LORA_CONFIG = LoraConfig(
    r=16,  # rank
    lora_alpha=32,  # scaling factor
    target_modules=["q_proj", "k_proj", "v_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
MODEL_CACHE_PATH = "/home/common/data/Big_Data/GenAI/llm_models"

print("=" * 80)
print(f"Using Device: {DEVICE}")
print(f"Final model will be saved to: {MODEL_PATH}")
print(f"LoRA adapters will be saved to: {ADAPTER_PATH}")
print(f"Finetuning Model: {BASE_MODEL}")
print(f"Using dataset from: {DATA_PATH}")
print(f"Model cache: {MODEL_CACHE_PATH}")
print("=" * 80)

In [ ]:
def generate_prompt_sql(input_question, context, output=""):
    """
    Generates a prompt for fine-tuning the LLM model for text-to-SQL tasks.

    Parameters:
        input_question (str): The input text or question to be converted to SQL.
        context (str): The schema or context in which the SQL query operates.
        output (str, optional): The expected SQL query as the output.

    Returns:
        str: A formatted string serving as the prompt for the fine-tuning task.
    """
    return f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. 

You must output the SQL query that answers the question.

### Input:
{input_question}

### Context:
{context}

### Response:
{output}"""

In [ ]:
def setup_model_and_tokenizer(base_model_id: str):
    """Downloads / Loads the pre-trained model in NF4 datatype and tokenizer based on the given base model ID for training."""
    local_model_id = base_model_id.replace("/", "--")
    local_model_path = os.path.join(MODEL_CACHE_PATH, local_model_id)
    print(f"local model path is: {local_model_path}")

    try:
        model = AutoModelForCausalLM.from_pretrained(
            local_model_path,
            load_in_low_bit="nf4",
            optimize_model=False,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )
    except OSError as e:
        print(e)
        sys.exit()
        logging.info(
            f"Model not found locally. Downloading {base_model_id} to cache..."
        )
        model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            load_in_low_bit="nf4",
            optimize_model=False,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )

    try:
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(local_model_path)
        else:
            tokenizer = AutoTokenizer.from_pretrained(local_model_path)
    except OSError:
        logging.info(
            f"Tokenizer not found locally. Downloading tokenizer for {base_model_id} to cache..."
        )
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(base_model_id)
        else:
            tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    tokenizer.pad_token_id = 0
    tokenizer.padding_side = "left"
    return model, tokenizer

In [17]:
import torch
class FineTuner:
    """A class to handle the fine-tuning of LLM models."""

    def __init__(self, base_model_id: str, model_path: str, device: torch.device):
        """
        Initialize the FineTuner with base model, model path, and device.

        Parameters:
            base_model_id (str): Id of pre-trained model to use for fine-tuning.
            model_path (str): Path to save the fine-tuned model.
            device (torch.device): Device to run the model on.
        """
        self.base_model_id = base_model_id
        self.model_path = model_path
        self.device = device
        self.model, self.tokenizer = setup_model_and_tokenizer(base_model_id)


    def tokenize_data(
        self, data_points, add_eos_token=True, train_on_inputs=False, cutoff_len=512
    ) -> dict:
        """
        Tokenizes dataset of SQL related data points consisting of questions, context, and answers.

        Parameters:
            data_points (dict): A batch from the dataset containing 'question', 'context', and 'answer'.
            add_eos_token (bool): Whether to add an EOS token at the end of each tokenized sequence.
            cutoff_len (int): The maximum length for each tokenized sequence.

        Returns:
            dict: A dictionary containing tokenized 'input_ids', 'attention_mask', and 'labels'.
        """
        try:
            question = data_points["question"]
            context = data_points["context"]
            answer = data_points["answer"]
            if train_on_inputs:
                user_prompt = generate_prompt_sql(question, context)
                tokenized_user_prompt = self.tokenizer(
                    user_prompt,
                    truncation=True,
                    max_length=cutoff_len,
                    padding=False,
                    return_tensors=None,
                )
                user_prompt_len = len(tokenized_user_prompt["input_ids"])
                if add_eos_token:
                    user_prompt_len -= 1

            combined_text = generate_prompt_sql(question, context, answer)
            tokenized = self.tokenizer(
                combined_text,
                truncation=True,
                max_length=cutoff_len,
                padding=False,
                return_tensors=None,
            )
            if (
                tokenized["input_ids"][-1] != self.tokenizer.eos_token_id
                and add_eos_token
                and len(tokenized["input_ids"]) < cutoff_len
            ):
                tokenized["input_ids"].append(self.tokenizer.eos_token_id)
                tokenized["attention_mask"].append(1)
            tokenized["labels"] = tokenized["input_ids"].copy()
            if train_on_inputs:
                tokenized["labels"] = [-100] * user_prompt_len + tokenized["labels"][
                    user_prompt_len:
                ]

            return tokenized
        except Exception as e:
            logging.error(
                f"Error in batch tokenization: {e}, Line: {e.__traceback__.tb_lineno}"
            )
            raise e

    def prepare_data(self, data, val_set_size=100) -> Dataset:
        """Prepare training and validation datasets."""
        try:
            train_val_split = data["train"].train_test_split(
                test_size=val_set_size, shuffle=True, seed=42
            )
            train_data = train_val_split["train"].shuffle().map(self.tokenize_data)
            val_data = train_val_split["test"].shuffle().map(self.tokenize_data)
            return train_data, val_data
        except Exception as e:
            logging.error(
                f"Error in preparing data: {e}, Line: {e.__traceback__.tb_lineno}"
            )
            raise e

    def train_model(self, train_data, val_data, training_args):
        """
        Fine-tune the model with the given training and validation data.

        Parameters:
            train_data (Dataset): Training data.
            val_data (Optional[Dataset]): Validation data.
            training_args (TrainingArguments): Training configuration.
        """
        try:
            self.model = self.model.to(self.device)
            self.model = prepare_model(self.model)
            self.model = get_peft_model(self.model, LORA_CONFIG)
            trainer = Trainer(
                model=self.model,
                train_dataset=train_data,
                eval_dataset=val_data,
                args=training_args,
                data_collator=DataCollatorForSeq2Seq(
                    self.tokenizer,
                    pad_to_multiple_of=8,
                    return_tensors="pt",
                    padding=True,
                ),
            )
            self.model.config.use_cache = False
            trainer.train()
            self.model.save_pretrained(self.model_path)
        except Exception as e:
            logging.error(f"Error in model training: {e}")

    def finetune(self, data_path, training_args):
        """
        Execute the fine-tuning pipeline.

        Parameters:
            data_path (str): Path to the data for fine-tuning.
            training_args (TrainingArguments): Training configuration.
        """
        try:
            data = load_dataset(data_path)
            train_data, val_data = self.prepare_data(data)
            self.train_model(train_data, val_data, training_args)
        except KeyboardInterrupt:
            print("Interrupt received, saving model...")
            self.model.save_pretrained(f"{self.model_path}_interrupted")
            print(f"Model saved to {self.model_path}_interrupted")
        except Exception as e:
            logging.error(f"Error in fintuning: {e}")

ModuleNotFoundError: No module named 'torch'

In [18]:
lets_finetune()

NameError: name 'lets_finetune' is not defined